In [1]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
def get_Job_Title(soup):
    jobs = soup.find_all('div',class_='row1')
    a_filt = [a for a in soup.find_all('a', class_='title') if a.get('href') is not None]
    job_title = [job.get('title') for job in a_filt]
    return job_title

In [3]:
#Company Name
def get_Company_Name(soup):
    company_span = soup.find_all('span',class_= 'comp-dtls-wrap')
    a_tags = [a for a in [a_tag.find('a') for a_tag in company_span] if a is not None]
    company_name = [atag.get('title') for atag in a_tags]
    return company_name

In [4]:
def get_Job_Location(soup):
    loc = 'locWdth'
    loct_tag = soup.find_all('span',{'class':loc})
    location = [loc.text for loc in loct_tag]
    return location

In [5]:
def get_Job_Url(soup):
    jobs = soup.find_all('div',class_='row1')
    a_filt = [a for a in soup.find_all('a', class_='title') if a.get('href') is not None]
    job_url = [url.get('href') for url in a_filt]
    return job_url

In [6]:
def get_info():
    my_info = {
        'Job Title': get_Job_Title(soup),
        'Company Name': get_Company_Name(soup),
        'Location':get_Job_Location(soup),
        'Link':get_Job_Url(soup)
    }
    return my_info

In [9]:
url = 'https://www.naukri.com/'
driver = webdriver.Chrome()
search = input("Enter skills / designations / companies:")
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(5)
job_search = driver.find_element(By.CLASS_NAME,'suggestor-input ')
job_search.send_keys(search)
driver.find_element(By.CLASS_NAME, 'qsbSubmit').click()
time.sleep(5)

data_list = []
for i in range(5):
    if i!=0:
        c = i+1
        d = f'//*[@id="lastCompMark"]/div[2]/a[{c}]'
        driver.find_element(By.XPATH,d).click()
    scroll_count = 10
    # Scroll down the page multiple times
    for _ in range(scroll_count):
        driver.execute_script("window.scrollBy(0, 535);")
        time.sleep(3)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    data = get_info()
    data_list.append(data)
df = pd.DataFrame({
    'Job Title': [item for sublist in [d['Job Title'] for d in data_list] for item in sublist],
    'Company Name': [item for sublist in [d['Company Name'] for d in data_list] for item in sublist],
    'Location': [item for sublist in [d['Location'] for d in data_list] for item in sublist],
    'Link': [item for sublist in [d['Link'] for d in data_list] for item in sublist]
})

# Save to a CSV file
df.to_csv('Naukri_com.csv',index=None)

Enter skills / designations / companies: Data Science
